In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm_notebook
import statsmodels.stats.multitest as ssm

# Описание используемых данных
Данные для этой задачи взяты из исследования, проведенного в Stanford School of Medicine. В исследовании была предпринята попытка выявить набор генов, которые позволили бы более точно диагностировать возникновение рака груди на самых ранних стадиях.

В эксперименте принимали участие 24 человек, у которых не было рака груди (normal), 25 человек, у которых это заболевание было диагностировано на ранней стадии (early neoplasia), и 23 человека с сильно выраженными симптомами (cancer).

Ученые провели секвенирование биологического материала испытуемых, чтобы понять, какие из этих генов наиболее активны в клетках больных людей.

Секвенирование — это определение степени активности генов в анализируемом образце с помощью подсчёта количества соответствующей каждому гену РНК.

В данных для этого задания вы найдете именно эту количественную меру активности каждого из 15748 генов у каждого из 72 человек, принимавших участие в эксперименте.

Вам нужно будет определить те гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.

Кроме того, вам нужно будет оценить не только статистическую, но и практическую значимость этих результатов, которая часто используется в подобных исследованиях.

Диагноз человека содержится в столбце под названием "Diagnosis".

In [2]:
data = pd.read_csv('gene_high_throughput_sequencing.csv')
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


# Практическая значимость изменения
Цель исследований — найти гены, средняя экспрессия которых отличается не только статистически значимо, но и достаточно сильно. В экспрессионных исследованиях для этого часто используется метрика, которая называется fold change (кратность изменения). Определяется она следующим образом:

![formula.PNG](formula.PNG)
где C, T — средние значения экспрессии гена в control и treatment группах соответственно. По сути, fold change показывает, во сколько раз отличаются средние двух выборок.


# Инструкции к решению задачи
Задание состоит из трёх частей. Если не сказано обратное, то уровень значимости нужно принять равным 0.05.

In [3]:
def write_answer(answer, num):
    file = 'answer_'+str(num)+'.txt'
    with open(file, 'w') as f:
        f.write(str(answer))

**Часть 1: применение t-критерия Стьюдента**
В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

* для групп normal (control) и early neoplasia (treatment)
* для групп early neoplasia (control) и cancer (treatment)

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.

In [4]:
alpha = 0.05
genes = data.drop(['Patient_id', 'Diagnosis'],axis=1).columns

In [5]:
T_test_1 = []

for gen in tqdm_notebook(genes):
    first_sample = data.loc[data.Diagnosis=='normal',gen]
    second_sample = data.loc[data.Diagnosis=='early neoplasia',gen]
    stat = stats.ttest_ind(first_sample, second_sample, equal_var = False)
    T_test_1.append( [gen, stat[0], stat[1]] )
    
T_test_1_res = pd.DataFrame(columns = ['gen', 't_stat', 'p_value'], data = T_test_1)


In [6]:
answer_1 = (T_test_1_res.p_value < alpha).sum()
write_answer(answer_1, 1)

In [7]:
print('Кол-во отвергнутых гипотез первого теста:', answer_1)

Кол-во отвергнутых гипотез первого теста: 1575


In [8]:
T_test_2 = []

for gen in tqdm_notebook(genes):
    first_sample = data.loc[data.Diagnosis=='early neoplasia',gen]
    second_sample = data.loc[data.Diagnosis=='cancer',gen]
    stat = stats.ttest_ind(first_sample, second_sample, equal_var = False)
    T_test_2.append( [gen, stat[0], stat[1]] )
    
T_test_2_res = pd.DataFrame(columns = ['gen', 't_stat', 'p_value'], data = T_test_2)

In [9]:
answer_2 = (T_test_2_res.p_value < alpha).sum()
write_answer(answer_2, 2)

In [10]:
print('Кол-во отвергнутых гипотез второго теста:',answer_2)

Кол-во отвергнутых гипотез второго теста: 3490


**Часть 2: поправка методом Холма**

Для этой части задания вам понадобится модуль multitest из statsmodels.

import statsmodels.stats.multitest as smm

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.5.

**Обратите внимание, что**

применять поправку на множественную проверку нужно ко всем значениям достигаемых уровней значимости, а не только для тех, которые меньше значения уровня доверия.
при использовании поправки на уровне значимости 0.025 меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом 0.025, а не 0.05)!

In [11]:
reject_1, p_corrected_1, a1_1, a2_1 = ssm.multipletests(T_test_1_res.p_value, alpha = 0.025, method = 'holm') 
T_test_1_res['p_corrected'] = p_corrected_1
T_test_1_res['reject'] = reject_1
T_test_1_res.reject.sum()

2

In [12]:
reject_2, p_corrected_2, a1_2, a2_2 = ssm.multipletests(T_test_2_res.p_value, alpha = 0.025, method = 'holm') 
T_test_2_res['p_corrected'] = p_corrected_2
T_test_2_res['reject'] = reject_2
T_test_2_res.reject.sum()

79

In [13]:
def fold_change(C,T, limit = 1.5):
    if T > C:
        fc = T / C
    elif T < C:
        fc = -C / T
    
    return  fc, np.abs(fc) > limit

In [14]:
FC_stat_1 = []
for gen in tqdm_notebook(genes):
    normal_mean = data.loc[data.Diagnosis=='normal',gen].mean()
    en_mean = data.loc[data.Diagnosis=='early neoplasia',gen].mean()
    fc = fold_change(normal_mean,en_mean, limit = 1.5)
    FC_stat_1.append( [fc[0],fc[1]] )
T_test_1_res['fold_change_stat'] = np.array(FC_stat_1)[:,0]
T_test_1_res['fc_more_limit'] = np.array(FC_stat_1)[:,1]

In [15]:
answer_3 = int(T_test_1_res[T_test_1_res.reject].fc_more_limit.sum())
write_answer(answer_3, 3)

In [16]:
print('Кол-во отвергнутых гипотез первого теста:',answer_3)

Кол-во отвергнутых гипотез первого теста: 2


In [17]:
FC_stat_2 = []
for gen in tqdm_notebook(genes):
    en_mean = data.loc[data.Diagnosis=='early neoplasia',gen].mean()
    cancer_mean = data.loc[data.Diagnosis=='cancer',gen].mean()
    fc = fold_change(en_mean,cancer_mean, limit = 1.5)
    FC_stat_2.append( [fc[0],fc[1]] )
T_test_2_res['fold_change_stat'] = np.array(FC_stat_2)[:,0]
T_test_2_res['fc_more_limit'] = np.array(FC_stat_2)[:,1]

In [18]:
answer_4 = int(T_test_2_res[T_test_2_res.reject].fc_more_limit.sum())
write_answer(answer_4, 4)

In [19]:
print('Кол-во отвергнутых гипотез второго теста:',answer_4)

Кол-во отвергнутых гипотез второго теста: 77


**Часть 3: поправка методом Бенджамини-Хохберга**
Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от H_0, когда они есть, и будут чаще отклонять H_0, когда отличий нет).

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5.

In [20]:
reject_1bh, p_corrected_1bh, a1_1bh, a2_1bh = ssm.multipletests(T_test_1_res.p_value, alpha = 0.025, method = 'fdr_bh') 
T_test_1_res['p_corrected_bh'] = p_corrected_1bh
T_test_1_res['reject_bh'] = reject_1bh
T_test_1_res.reject_bh.sum()

4

In [21]:
answer_5 = int(T_test_1_res[T_test_1_res.reject_bh].fc_more_limit.sum())
write_answer(answer_5, 5)

In [22]:
print('Кол-во отвергнутых гипотез первого теста:',answer_5)

Кол-во отвергнутых гипотез первого теста: 4


In [23]:
reject_2bh, p_corrected_2bh, a1_2bh, a2_2bh = ssm.multipletests(T_test_2_res.p_value, alpha = 0.025, method = 'fdr_bh') 
T_test_2_res['p_corrected_bh'] = p_corrected_2bh
T_test_2_res['reject_bh'] = reject_2bh
T_test_2_res.reject_bh.sum()

832

In [24]:
answer_6 = int(T_test_2_res[T_test_2_res.reject_bh].fc_more_limit.sum())
write_answer(answer_6, 6)

In [25]:
print('Кол-во отвергнутых гипотез второго теста:',answer_6)

Кол-во отвергнутых гипотез второго теста: 524
